# (제목변경) 자동화 코드 주제

### 디렉토리 설정
- 아래 각 변수에 해당하는 알맞은 경로를 찾아 입력해주세요.
- 경로 주소는 '.../.../...' 형식을 따라주세요.

In [ ]:
class directory:
    dropbox_dir = 'C:/Dropbox (주식회사매드업)'
    # 드롭박스 폴더 위치경로
    
    raw_dir = dropbox_dir + ''
    # raw_data가 적재되어 있는 폴더 위치경로
    
    download_dir = 'C:/Users/MADUP/Downloads'
    # 최종 결과파일을 다운받을 폴더 위치경로

dr = directory()

### 날짜 설정
- 보통 작일자 기준으로 요청주신 기간의 데이터를 출력하게 됩니다.
- 날짜 조정이 필요한 경우 아래 변수 값을 수정하여 출력해주세요.

In [ ]:
class report_date:
    today = datetime.date.today()
    # 오늘 날짜 ex> 2022-09-14
    
    # today = datetime.date(year = 2022, month = 8, day = 12)
    # 다른 일자를 기준으로 추출하고 싶은 경우 위 코드 수정 및 주석해제
    
    day_1 = today - datetime.timedelta(1)
    # 전일자 ex> 2022-09-13
    
    start_day = datetime.date(year=day_1.year, month=day_1.month, day=1)
    # 금월 첫 1일자 ex> 2022-09-01
    
    yearmonth = day_1.strftime('%Y%m')
    # 년월 ex> 202209
    
    month_name = str(day_1.month) + '월'
    # 금월 ex> 9월

import datetime
rdate = report_date()

### 코드
- 전처리 자동화를 위한 코드로 임의 수정을 하실 경우 복사본 생성하여 진행해주시길 바랍니다.
- 혹 아래 이미지와 같은 형식의 오류가 발생할 경우 `!pip install [패키지명]`형식의 코드 작성하여 실행해주시면 해당 패키지가 설치됩니다.
    - <img src = 'https://ifh.cc/g/ymACFT.png' align='left'>

- 이외의 오류가 발생하거나 문제가 해결되지 않는 경우, 데컨팀에 문의주세요 :)

In [ ]:
# 패키지 설치 오류 방지 코드 (초기 설치 후 지우셔도 됩니다.)
!pip install pyarrow
!pip install datetime

In [ ]:
# 자동화 코드 복사 붙여넣기

---

### 코드 작성 시 유의사항
- directory, date 관련 변수는 가능한 class 내에 추가해두고 전역변수로 사용하지 않도록 한다.
- 자동화 코드 내에서 사용되는 directory, date 관련 변수는 가능한 인수로 전달받아 매개변수로써 사용되도록 작성한다.
- 추가로 커스텀 유지보수를 위해 제공하고자 하는 전역변수는 상단에 별도로 기재한다.

[아래는 예시 코드입니다.]

In [ ]:
media_source = ['appier_int']
raw_dir = dr.dropbox_dir + '/광고사업부/4. 광고주/잡코리아/!자동화리포트/raw_data/appsflyer_prism'
result_dir = dr.download_dir

In [ ]:
import pandas as pd
import os
import numpy as np
import pyarrow as pa
import pyarrow.csv as pacsv
import datetime


def deduction_data(raw_dir, required_date):
    dtypes = {
        'attributed_touch_type': pa.string(),
        'attributed_touch_time': pa.string(),
        'install_time': pa.string(),
        'event_time': pa.string(),
        'event_name': pa.string(),
        'country_code': pa.string(),
        'language': pa.string(),
        'media_source': pa.string(),
        'carrier': pa.string(),
        'os_version': pa.string(),
        'app_version': pa.string(),
        'sdk_version': pa.string(),
        'device_model': pa.string(),
        'campaign': pa.string(),
        'campaign_id': pa.string(),
        'adset': pa.string(),
        'adset_id': pa.string(),
        'ad': pa.string(),
        'ad_id': pa.string(),
        'advertising_id': pa.string(),
        'appsflyer_id': pa.string(),
        'idfa': pa.string(),
        'idfv': pa.string(),
        'android_id': pa.string(),
        'platform': pa.string(),
        'channel': pa.string(),
        'keywords': pa.string(),
        'is_retargeting': pa.string(),
        'is_primary_attribution': pa.string()
    }
    index_columns = list(dtypes.keys())
    convert_ops = pacsv.ConvertOptions(column_types=dtypes, include_columns=index_columns)
    ro = pacsv.ReadOptions(block_size=10 << 20)
    table_list = []

    date_check = required_date.strftime('%Y%m')
    start_date = required_date.replace(day=1).strftime('%Y%m%d')
    end_date = required_date.strftime('%Y%m%d')

    files = os.listdir(raw_dir)
    files = [f for f in files if '.csv' in f and str(f)[-12:-6] == date_check]
    raw_files = [f for f in files if
                 (int(str(f)[-12:-4]) >= int(start_date)) & (int(str(f)[-12:-4]) <= int(end_date))]

    for f in raw_files:
        try:
            temp = pacsv.read_csv(raw_dir + '/' + f, convert_options=convert_ops, read_options=ro)
            table_list.append(temp)
        except Exception as e:
            print(f)
            print(e)
    table = pa.concat_tables(table_list)
    raw_df = table.to_pandas()
    raw_df = raw_df.loc[raw_df['media_source'].isin(media_source)]
    raw_df[['attributed_touch_time', 'install_time', 'event_time']] = raw_df[
        ['attributed_touch_time', 'install_time', 'event_time']].apply(pd.to_datetime)

    return raw_df


def deduction_calculate(raw_dir, required_date, df, file_name):
    df['ITET'] = df['event_time'] - df['install_time']
    df['Date'] = df['event_time'].apply(lambda x: x.date())

    df[['con1_event', 'con2_country', 'con3_language', 'con4_carrier', 'con5_device', 'con6_osv', 'con7_appv',
        'con8_sdkv', 'con9_over3(reattr)', 'con10_lastm']] = 0

    detarget_event = ['signup_all', 'business_signup_complete', 'job_apply_complete_mobile',
                      'job_apply_complete_homepage', 'resume_complete']
    df.loc[df['event_name'].isin(detarget_event), 'con1_event'] = 1
    # con1_event

    df.loc[df['country_code'] != 'KR', 'con2_country'] = 1
    # con2_country

    df.loc[(df['platform'] == 'android') & (df['language'] != '한국어'), 'con3_language'] = 1
    # con3_language

    target_carrier = [x.lower() for x in ['SK Telecom', 'SKTelecom', 'SKT', 'KOR SK Telecom', 'KT', 'LGU+', 'LG U+', 'olleh']]
    df['carrier'] = df['carrier'].str.lower()
    df.loc[(df['platform'] == 'android') & ~(df['carrier'].isin(target_carrier)), 'con4_carrier'] = 1
    # con4_carrier

    detarget_device = [x.lower() for x in
                       ['allwinner', 'GIONEE', 'HONOR', 'HUAWEI', 'Infinix', 'iQOO', 'Itel', 'OnePlus', 'OPPO', 'POCO',
                        'realme', 'Redmi', 'sharp', 'TECNO', 'TGnCo', 'tufen', 'umidigi', 'vivo', 'Xiaomi', 'ZTE',
                        'SMARTISAN', 'Lenovo']]
    df['device_model_name'] = df['device_model'].apply(lambda x: x.split('::')[0]).str.lower()
    df.loc[df['device_model_name'].isin(detarget_device), 'con5_device'] = 1
    # con5_device

    def os_version(row):
        if row['platform'] == 'android':
            if int(row['os_version'].split('.')[0]) < 9:
                return 1
            else:
                return 0
        elif row['platform'] == 'ios':
            if int(row['os_version'].split('.')[0]) < 14:
                return 1
            else:
                return 0

    df.loc[:, 'con6_osv'] = df.apply(os_version, axis=1)
    # con6_osv

    def app_version(row):
        if row['platform'] == 'android':
            if int(row['app_version'].replace('.', '')) < 333:
                return 1
            else:
                return 0
        elif row['platform'] == 'ios':
            if int(row['app_version'].replace('.', '')) < 364:
                return 1
            else:
                return 0

    df.loc[:, 'con7_appv'] = df.apply(app_version, axis=1)
    # con7_appv

    def app_version(row):
        if row['platform'] == 'android':
            if int(row['sdk_version'].replace('.', '').replace('v', '')) < 614:
                return 1
            else:
                return 0
        elif row['platform'] == 'ios':
            if int(row['sdk_version'].replace('.', '').replace('v', '')) < 626:
                return 1
            else:
                return 0

    df.loc[:, 'con8_sdkv'] = df.apply(app_version, axis=1)
    # con8_sdkv

    def over3_check_func(df):
        over3_check_df = df[['advertising_id', 'idfa', 'event_name', 'event_time']].sort_values(
            ['advertising_id', 'idfa', 'event_name', 'event_time'])
        over3_check_df = over3_check_df.drop(over3_check_df[over3_check_df['idfa'] == ''].index).reset_index(drop=True)
        over3_comp_df = over3_check_df.rename(
            columns={'advertising_id': 'c_advertising_id', 'idfa': 'c_idfa', 'event_name': 'c_event_name',
                     'event_time': 'c_event_time'}).iloc[2:, :].reset_index(drop=True)
        over3_check = pd.concat([over3_check_df, over3_comp_df], axis=1)

        def over3_checking(row):
            if (row['advertising_id'] == row['c_advertising_id']) & (row['idfa'] == row['c_idfa']) & (
                    row['event_name'] == row['c_event_name']):
                return (row['c_event_time'] - row['event_time']).total_seconds()
            else:
                return datetime.timedelta(0).total_seconds()

        over3_check['time_diff'] = over3_check.apply(over3_checking, axis=1)
        deductioin_idfa = over3_check.loc[
            (over3_check['time_diff'] < 60) & (over3_check['time_diff'] > 0)].idfa.unique()
        return deductioin_idfa

    deductioin_idfa = over3_check_func(df)
    df.loc[df['idfa'].isin(deductioin_idfa), 'con9_over3(reattr)'] = 1
    # con9_over3(reattr)

    lastm_date = required_date.replace(day=1) - datetime.timedelta(days=1)
    lastm_df = deduction_data(raw_dir, lastm_date)
    lastm_deductioin_idfa = over3_check_func(lastm_df)
    df.loc[df['idfa'].isin(lastm_deductioin_idfa), 'con10_lastm'] = 1
    # con10_lastm

    df.index = range(0, len(df))
    fraud_columns = ['con1_event', 'con2_country', 'con3_language', 'con4_carrier', 'con5_device', 'con6_osv',
                     'con7_appv', 'con8_sdkv', 'con9_over3(reattr)', 'con10_lastm']
    df.loc[df[fraud_columns].values.sum(axis=1) >= 1, 'is_fraud'] = 1
    df['is_fraud'] = df['is_fraud'].fillna(0)
    result_df = df[
        ['ITET', 'Date', 'is_fraud', 'attributed_touch_type', 'event_name', 'media_source', 'campaign', 'adset',
         'adset_id',
         'carrier', 'language', 'country_code', 'platform', 'os_version', 'app_version', 'sdk_version', 'device_model',
         'device_model_name',
         'con1_event', 'con2_country', 'con3_language', 'con4_carrier', 'con5_device', 'con6_osv', 'con7_appv',
         'con8_sdkv', 'con9_over3(reattr)',
         'con10_lastm', 'idfa', 'advertising_id']]

    result_df.to_csv(result_dir + '/' + file_name, index=False, encoding='utf-8-sig')
    print('download_success')


required_date = rdate.day_1
df = deduction_data(raw_dir, rdate.day_1)
file_name = f'{rdate.month_name} 디덕션 적용.csv'
deduction_calculate(raw_dir, required_date, df, file_name)